In [16]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers

import numpy as np

In [2]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [3]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(mnist)

In [4]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [5]:
linear = Sequential()
linear.add(Flatten(input_shape=x_train.shape[1:]))
linear.add(Dense(units=10, activation='softmax'))

In [17]:
small_cnn = Sequential()

small_cnn.add(Conv2D(64, (5, 5), activation='relu', input_shape=x_train.shape[1:], padding="same"))
small_cnn.add(Conv2D(64, (5, 5), activation='relu',  padding="same", ))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), activation='relu',  padding="same"))
small_cnn.add(Conv2D(64, (3, 3), activation='relu',  padding="same"))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), activation='relu',  padding="same"))
small_cnn.add(Conv2D(64, (3, 3), activation='relu',  padding="same"))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Flatten())

small_cnn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
small_cnn.add(Dense(units=64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))

small_cnn.add(Dense(units=10, activation='softmax'))

In [18]:
#model = linear
model = small_cnn

In [19]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [20]:
n_train    = 600
batch_size = 128
epochs     = 40

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:600], y_test[:600]))

Train on 600 samples, validate on 600 samples
Epoch 1/40
600/600 [==============================] - 19s - loss: 4.0732 - acc: 0.1067 - val_loss: 4.0137 - val_acc: 0.1867
Epoch 2/40
600/600 [==============================] - 18s - loss: 3.9803 - acc: 0.1433 - val_loss: 3.9223 - val_acc: 0.2017
Epoch 3/40
600/600 [==============================] - 18s - loss: 3.8847 - acc: 0.1650 - val_loss: 3.8235 - val_acc: 0.2383

In [21]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', acc)

Test accuracy: 0.9293
